# 🕵🏼 **6. Óra:** Lineáris rendszer szabályozása

## 🎯 Küldetés  

- Ismétlés (diszkrét deriválás és trapéz módszer)
- Bevezetés a PID szabályozásba
- Szabályozási feladat megoldása

---

## Hasznos linkek  
🔍 [Lényegretörő bevezetés a PID szabályozásba](https://www.youtube.com/watch?v=UR0hOmjaHp0) 

🔍 [Egy egyszerű példa](https://www.youtube.com/watch?v=XfAt6hNV8XM)

## Ismétlés

### Deriválás diszkrét időben
Legyen $ x[n] $ egy diszkrét időben definiált jel, és legyen $ \Delta t $ a mintavételezési idő. A diszkrét időbeli derivált vagy különbségi hányados a következőképpen definiálható:

$$
x'[n] = \frac{x[n] - x[n-1]}{\Delta t}
$$

A már jól ismért differencia számító függvényt fogjuk alkalmazni:
```python
def deriv(function, time_step):
    derivative = []
    for i in range(1, len(function)):
        derivative.append((function[i] - function[i-1]) / time_step)
    return derivative
```

### Numerikus integrálás trapéz módszerrrel
Az interpoláló függvény egy egyenes vonal lehet (azaz egy 1-es fokú polinom), amely áthalad az $ (a, f(a)) $ és $ (b, f(b)) $ pontokon. Ezt hívjuk **trapéz szabálynak**, amely a következőképpen van megadva:

$$
\int_a^b f(x) \, dx \approx (b - a) \left( \frac{f(a) + f(b)}{2} \right)
$$

<center>
  <img src="Integration_trapezoid.png" width="500" style="background-color: white;" />
</center>

Ezt már megtanultuk korábban implementálni a következő módon, egység hosszú intervallumok esetén:
```python
def trapezoidal_integral(function, time_step):
    integral = [0]
    comulative_sum = 0
    for i in range(1, len(function)):
        comulative_sum += time_step * (function[i-1] + function[i]) / 2
        integral.append(comulative_sum)
    return integral
```

## Bevezetés a PID szabályozásba

A PID (Arányos-Integráló-Deriváló) szabályozó az egyik legelterjedtebb szabályozási módszer az iparban és a mérnöki gyakorlatban. Alapelve, hogy a rendszer **eltérését** (hibáját) a kívánt, ún. **referencia értéktől** három komponens segítségével kompenzálni tudjuk, így a számunkra megfelelő állapotba juttatva a rendszert:  

1. **Arányos (P) tag:** A jelenlegi hibaérték alapján határozza meg a szabályozó válaszát. Minél nagyobb az eltérés, annál nagyobb a beavatkozás.
2. **Integráló (I) tag:** A múltbeli hibák összegzésével korrigálja az esetleges állandósult hibát.
3. **Deriváló (D) tag:** A hiba változási sebessége alapján reagál, csökkentve a túllövést és a rendszer ingadozását.

A PID szabályozó kimenete az alábbi egyenlet szerint számítható, ez lesz a beavatkozó jelünk, amit kiadunk a rendszerre:  

$$
u(t) = K_p e(t) + K_i \int e(t) dt + K_d \frac{de(t)}{dt}
$$

ahol:  
- $ e(t) $ a hiba (az eltérés a kívánt értéktől),  
- $ K_p $, $ K_i $ és $ K_d $ a szabályozó erősítési tényezői (súlyok), ezeket mi hangoljuk be, hogy elérjük a kívánt viselkedést.

<center>
  <img src="PID_control.png" width="500" style="background-color: white;" />
</center>

### A PID szabályozás előnyei
- Képes minimalizálni a szabályozási hibát.
- Rugalmas és sokféle rendszerhez alkalmazható.
- Az integráló tag kiküszöböli az állandósult hibát, míg a deriváló tag csökkenti az ingadozást (túllövést).

### Hol használják?
A PID szabályozót széles körben alkalmazzák például:
- Robotkarok pozíciószabályozásában,
- Hőmérséklet-szabályozásban,
- Motorok fordulatszám-szabályozásában,
- Drónok egyensúlyozásában.


## Első szabályozási feladatunk: inverz inga

<center>
  <img src="pendulum.png" width="500" style="background-color: white;" />
</center>

🔍 [Inverz inga szimulátor](https://codepen.io/oscarsaharoy/full/LYbmVma)

### Importok

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Az ismerős függvények

In [ ]:
def deriv(function, time_step):
    derivative = []
    for i in range(1, len(function)):
        derivative.append((function[i] - function[i-1]) / time_step)
    return derivative

def trapezoidal_integral(function, time_step):
    integral = [0]
    cumulative_sum = 0
    for i in range(1, len(function)):
        cumulative_sum += time_step * (function[i-1] + function[i]) / 2
        integral.append(cumulative_sum)
    return integral

In [ ]:
# Rendszerparaméterek
dt = 0.02  # Időlépés
g = 9.81   # Gravitáció
l = 0.5    # Inga hossza
m = 0.1    # Inga tömege
M = 1.0    # Cart mass
b = 0.1    # Friction

# PID Gains
Kp = 50     # Proportional gain
Ki = 5      # Integral gain
Kd = 10     # Derivative gain

# Simulation setup
T = 5  # Total simulation time
N = int(T / dt)

# Initial conditions
theta = np.pi + 0.1  # Initial angle (small deviation)
x = 0  # Cart position
x_dot = 0  # Cart velocity

# Data logging
theta_list = [theta]  # Store theta values
error_list = [theta - np.pi]  # Store error (theta - desired theta)
control_list = []  # Store control force values

# Simulation loop
for _ in range(N - 1):
    # Compute control force (PID control)
    error = theta_list[-1] - np.pi
    error_list.append(error)
    
    # Compute PID terms
    P_term = Kp * error
    I_term = Ki * trapezoidal_integral(error_list, dt)[-1]  # Integral of error
    D_term = Kd * (0 if len(error_list) == 1 else deriv(error_list, dt)[-1])  # Derivative of error
    
    u = -(P_term + I_term + D_term)  # Control force applied to cart

    # Dynamics (simplified)
    x_ddot = (u - b * x_dot) / M
    theta_ddot = (g / l) * np.sin(theta_list[-1]) + (x_ddot / l) * np.cos(theta_list[-1])

    # Integrate to get new state
    x_dot += x_ddot * dt
    x += x_dot * dt
    theta_list.append(theta_list[-1] + dt * (0 if len(theta_list) == 1 else deriv(theta_list, dt)[-1]))  # Update theta
    control_list.append(u)

# Compute derivatives after the loop
theta_dot_list = deriv(theta_list, dt)
theta_ddot_list = deriv(theta_dot_list, dt)

# Plot results
time_axis = np.linspace(0, T, N - 1)

plt.figure(figsize=(10, 5))

plt.subplot(2, 1, 1)
plt.plot(time_axis, theta_list[:-1], label="Theta (rad)")
plt.xlabel("Time [s]")
plt.ylabel("Pendulum Angle [rad]")
plt.title("Inverted Pendulum Control")
plt.legend()
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(time_axis, control_list, label="Control Force (N)", color='red')
plt.xlabel("Time [s]")
plt.ylabel("Control Input (N)")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()
